#### SET DIRECTORIES !!!

In [ ]:
data_directory="/path/to/data/directory"
directory="path/to/sens-sensor"
print(f"Data directory is {data_directory}")
print(f"Working directory is {directory}")

# ARAUS-extended dataset generation - preparing original ARAUS
This script adequates the augmented audios and the data csv from ARAUS dataset so that it is prepared to generate the new ARAUS-extended dataset.
responses.csv provided by ARAUS authors contains the data associated with the augmented soundscapes (participant answers, features of the audio, fold to which the audio belongs, base soundscape and masker used for the augmentation...).
However, we are included some new columns into the dataframe, so that it is complete and handy for our operations.
1) We are addind Pleasantness and Eventfulness values calculated from the participant answers punctuations --> 2 more columns
2) We are adding the wav gain that has to be applied to each digital signal to convert it to pressure signal in Pascals --> 1 more column

In [26]:
# Set the working directory
import os
os.chdir(directory)

Current Working Directory: /Users/amaiasagastimartinez/Desktop/SENS-Soundlights/code/sens-sensor


In [27]:
import numpy as np
import pandas as pd
from IPython.display import HTML
from maad.util import mean_dB
from maad.spl import pressure2leq

# Imports from this project
from development.lib.dataset_functions import load

## Declare inputs

In [28]:
# Path to original ARAUS dataset
path_csv=os.path.join(data_directory,"files/responses.csv")
# Path to save new adapted dataset
saving_path=os.path.join(data_directory,"files/responses_adapted.csv")

## Load ARAUS original csv file
Obtained directly from ARAUS repository

In [29]:
responses = pd.read_csv(os.path.join(directory, path_csv), dtype = {'participant':str})

## Calculate P and E

Ground truth labels refer to the actual, true, or correct values of the target variable (or labels) in a supervised machine learning task. In other words, these are the known outcomes or responses associated with the input data points. The purpose of ground truth labels is to provide a basis for training and evaluating machine learning models.

In [30]:
# Show images
image1_path = f"{data_directory}/images/PandE_axis.png"
image2_path = f"{data_directory}/images/PandE_formulas.png"
html_code1 = f'<img src="{image1_path}" width="500">'
html_code2 = f'<img src="{image2_path}" width="700">'
display(HTML(html_code1))
display(HTML(html_code2))



Weights for ISO pleasantness:
- Pleasant: 1
- Eventful: 0
- Chaotic: -sqrt(2)/2
- Vibrant: sqrt(2)/2
- Uneventful: 0
- Calm: sqrt(2)/2
- Annoying: -1
- Monotonous: -sqrt(2)/2

Weights for ISO eventfulness:
- Pleasant: 0
- Eventful: 1
- Chaotic: sqrt(2)/2
- Vibrant: sqrt(2)/2
- Uneventful: -1
- Calm: -sqrt(2)/2
- Annoying: 0
- Monotonous: -sqrt(2)/2

In [31]:
# Define attributes to extract from dataframes
attributes = ['pleasant', 'eventful', 'chaotic', 'vibrant', 'uneventful', 'calm', 'annoying', 'monotonous'] 
# Define weights for each attribute in attributes in computation of ISO Pleasantness
ISOPl_weights = [1,0,-np.sqrt(2)/2,np.sqrt(2)/2, 0, np.sqrt(2)/2,-1,-np.sqrt(2)/2] 
# Define weights for each attribute in attributes in computation of ISO Eventfulness
ISOEv_weights = [0,1,np.sqrt(2)/2,np.sqrt(2)/2, -1, -np.sqrt(2)/2,0,-np.sqrt(2)/2] 
# Copy 
responses_PE = responses.copy() 
# These are normalised ISO Pleasantness values (in [-1,1])
responses_PE['P_ground_truth'] = ((responses[attributes] * ISOPl_weights).sum(axis=1)/(4+np.sqrt(32))).values
# These are normalised ISO Eventfulness values (in [-1,1])
responses_PE['E_ground_truth'] = ((responses[attributes] * ISOEv_weights).sum(axis=1)/(4+np.sqrt(32))).values
print(responses_PE.shape, responses_PE)

(27255, 162)        participant  fold_r                          soundscape  \
0      ARAUS_00001      -1  R0091_segment_binaural_44100_1.wav   
1      ARAUS_00001       1  R0079_segment_binaural_44100_1.wav   
2      ARAUS_00001       1  R0056_segment_binaural_44100_2.wav   
3      ARAUS_00001       1  R0046_segment_binaural_44100_2.wav   
4      ARAUS_00001       1  R0092_segment_binaural_44100_1.wav   
...            ...     ...                                 ...   
27250  ARAUS_10005       0    R1007_segment_binaural_44100.wav   
27251  ARAUS_10005       0    R1006_segment_binaural_44100.wav   
27252  ARAUS_10005       0    R1008_segment_binaural_44100.wav   
27253  ARAUS_10005       0    R1007_segment_binaural_44100.wav   
27254  ARAUS_10005      -1  R0091_segment_binaural_44100_1.wav   

                       masker  smr  stimulus_index  time_taken  is_attention  \
0           silence_00001.wav    0               1      98.328             0   
1           silence_00001.wav    6

## Wav gains for each augmented soundscape

In ARAUS dataset responses.csv constitute the dataset of +25k augmented soundscapes labeled with psychoacoustic and acoustic parametres. Among these, we can find Leq_r, which constitutes the Leq of channel R for each audio.

In order to generate certain features (the ones we call "ARAUS features" as they aim to replicate the original ARAUS features), it is needed to know the gain or calibration factor that was applyied to the wav files (audios) in order to get the specified Leq. This linear gain (that converts wav to Peak-Pascals), one for each audio, is calculated in this section, and it must be stored. 

For the other two set of features (the ones we call "Freesound features" and for the CLAP embedding generation), the audios need to be coherent between each other in terms of energy, meaning that audios that were played with less volume, should have less amplitude than those who were played with higher volume. The factor that gives us this proportionate relation is the gain mentioned in the paragraph above. Therefore, this gain value is also needed for this set of features.

This gain is stored in the new csv, in a new column.

In [ ]:
# Path to folder containing original augmented soundscapes
audioFolderPath=os.path.join(data_directory,"soundscapes_augmented")
print(f"Audio folder path ", audioFolderPath)
# Prepare output dataframe
columns=responses_PE.columns
newDF=pd.DataFrame(columns=columns)
newDF.insert(loc=6, column='wav_gain', value=None)
# Go over all the audio files in the given directory 
count_clip=0
count_total=0
clipping=[]
# Go over each audio file
files = sorted(os.listdir(audioFolderPath))
files.sort()
for file in files:
    print(file)
    if file.endswith(".mp3") or file.endswith(".wav"):
        print("file ", file)
        print("count total ", count_total)
        # Find the row in responses.csv corresponding to current audio
        audio_path = os.path.join(data_directory, "soundscapes_augmented",file)
        file_split = file.split("_")
        file_fold = int(file_split[1])
        file_participant = "ARAUS_" + file_split[3]
        file_stimulus = int(file_split[5].split(".")[0])
        audio_info_aug = responses_PE[responses_PE["fold_r"] == file_fold]
        audio_info_aug = audio_info_aug[
            audio_info_aug["stimulus_index"] == file_stimulus
        ]
        audio_info_aug = audio_info_aug[
            audio_info_aug["participant"] == file_participant
        ]
        # Get the original Leq of this audio 
        true_Leq=audio_info_aug["Leq_R_r"].values[0]
        # Load the stereo audio file
        audio_r,fs=load(audio_path, wav_calib=1.0, ch=1)
        audio_l,fs=load(audio_path, wav_calib=1.0, ch=0)
        # Calculate gain from true Leq and "raw" Leq
        rawR_Leq=mean_dB(pressure2leq(audio_r, fs, 0.125))
        gain_dB=true_Leq-rawR_Leq
        gain=10**(gain_dB/20)
        # Add gain info
        audio_info_aug["wav_gain"]=gain
        # Add audio file name
        audio_info_aug["file"]=file.split(".")[0]
        newDF = pd.concat([newDF, audio_info_aug], ignore_index=True)
        # Prepare next iteration
        count_total=count_total+1

## Save new generated dataset

In [33]:
newDF.to_csv(os.path.join(data_directory,saving_path), index=False)